# Trainingspipeline 1

- ohne Bigramme
- vector_size: 300
- window: 3
- min_count: 2
- seed: 1
- CBOW

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [3]:
df = pd.read_csv('../../Korpus/Korpus/corpus_lemmatized.csv', sep=',', encoding='utf-8')

In [4]:
df.shape

(304129, 10)

In [5]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [6]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [7]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1825']
df4 = df_periods['1826-1850']
df5 = df_periods['1851-1875']
df6 = df_periods['1876-1900']
df7 = df_periods['1901-1925']
df8 = df_periods['1926-1950']
df9 = df_periods['1951-1985']
df10 = df_periods['1986-2000']
df11 = df_periods['2001-2021']

## Training von Word2Vec

In [8]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [9]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 3                         # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 0                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1

### Zeitraum 1: 1700-1750

In [10]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [11]:
%%time
sentences1 = tokenize_text(text1)         # Bereinigen, Tokenisieren und in Form bringen (Ziel: Liste von tokenisierten Sätzen)

Wall time: 16.1 s


In [12]:
print(sentences1[3])

['lungo', 'inospita', 'campagna', 'aggiungere', 'invidia', 'lato', 'manco', 'dire', 'anch', 'essere', 'teco', 'labbro', 'bianco', 'veggo', 'veneno', 'cor', 'stagno']


In [13]:
len(sentences1)

113328

In [14]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 32 s


In [15]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('movimento', 0.9195557832717896),
 ('assalto', 0.9172037243843079),
 ('macello', 0.9140099883079529),
 ('folla', 0.9122866988182068),
 ('lite', 0.9095157980918884),
 ('strepito', 0.9083541631698608),
 ('naviglio', 0.9074839949607849),
 ('fretta', 0.9032074809074402),
 ('disagio', 0.9030563235282898),
 ('combinazione', 0.9016652703285217)]

In [16]:
# trainiertes Modell speichern

w2v1.save(os.path.join('trained_models/Word2VecCBOW1', '1w2vcbow1.model'))

### Zeitraum 2: 1751-1800

In [17]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [18]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 18.2 s


In [19]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 31.4 s


In [20]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('usbergo', 0.9606921076774597),
 ('cadea', 0.9581971764564514),
 ('poter', 0.9576661586761475),
 ('laccio', 0.9556865096092224),
 ('crudeltà', 0.9548324942588806),
 ('cancello', 0.954310953617096),
 ('servaggio', 0.954087495803833),
 ('freno', 0.9539632797241211),
 ('morrà', 0.9538893103599548),
 ('cittadin', 0.9538716673851013)]

In [21]:
w2v2.save(os.path.join('trained_models/Word2VecCBOW1', '1w2vcbow2.model'))

### Zeitraum 3: 1801-1825

In [22]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [23]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 15.6 s


In [24]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 30.2 s


In [25]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('crudele', 0.9429351687431335),
 ('vendetta', 0.9414681792259216),
 ('immortale', 0.9382587671279907),
 ('ardore', 0.9342174530029297),
 ('riverenza', 0.9283024668693542),
 ('invidia', 0.9262387156486511),
 ('sdegno', 0.9259082674980164),
 ('furore', 0.9250084757804871),
 ('orrore', 0.9212765693664551),
 ('affannato', 0.9195654988288879)]

In [26]:
w2v3.save(os.path.join('trained_models/Word2VecCBOW1', '1w2vCBOW2.model'))

### Zeitraum 4: 1826-1850

In [27]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [28]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 15.9 s


In [29]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 29.5 s


In [30]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('vigore', 0.9458653330802917),
 ('senno', 0.9323484301567078),
 ('energia', 0.9289345741271973),
 ('ricchezza', 0.9266555309295654),
 ('felicità', 0.9247794151306152),
 ('ambizione', 0.9242504239082336),
 ('virt', 0.9233769774436951),
 ('violenza', 0.9231333136558533),
 ('odio', 0.9182305932044983),
 ('miseria', 0.9178458452224731)]

In [31]:
w2v4.save(os.path.join('trained_models/Word2VecCBOW1', '1w2vcbow4.model'))

### Zeitraum 5: 1851-1875

In [32]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [33]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 16.5 s


In [34]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 28.6 s


In [35]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('angoscia', 0.9387381672859192),
 ('sgomento', 0.9295408129692078),
 ('impeto', 0.9291709065437317),
 ('disperazione', 0.9290449023246765),
 ('sdegno', 0.9268385767936707),
 ('commozione', 0.9258841872215271),
 ('rimorso', 0.9201672673225403),
 ('raccapriccio', 0.9188809990882874),
 ('tenerezza', 0.9180452823638916),
 ('calma', 0.9147021174430847)]

In [36]:
w2v5.save(os.path.join('trained_models/Word2VecCBOW1', '1w2vcbow5.model'))

### Zeitraum 6: 1876-1900

In [37]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [38]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 17.7 s


In [39]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 28.4 s


In [40]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('inesprimibile', 0.9093572497367859),
 ('ineffabile', 0.8945512175559998),
 ('animare', 0.8928439617156982),
 ('sgomento', 0.8919458389282227),
 ('emozione', 0.8908723592758179),
 ('turbamento', 0.8892930746078491),
 ('palpito', 0.8844007849693298),
 ('malinconia', 0.8833568096160889),
 ('palpitare', 0.8823610544204712),
 ('volutt', 0.8812597990036011)]

In [41]:
w2v6.save(os.path.join('trained_models/Word2VecCBOW1', '1w2vcbow6.model'))

### Zeitraum 7: 1901-1925

In [42]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [43]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 17.5 s


In [44]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 28.8 s


In [45]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('sgomento', 0.9420453310012817),
 ('angoscia', 0.9415345191955566),
 ('impeto', 0.9207671880722046),
 ('gioia', 0.9123913645744324),
 ('dolcezza', 0.9096062183380127),
 ('spavento', 0.9065960049629211),
 ('commozione', 0.9058353900909424),
 ('tenerezza', 0.8996182680130005),
 ('sdegno', 0.8987729549407959),
 ('inquietudine', 0.8954268097877502)]

In [46]:
w2v7.save(os.path.join('trained_models/Word2VecCBOW1', '1w2vcbow7.model'))

### Zeitraum 8: 1926-1950

In [47]:
text8 = ''

for i in df8.lemmatized_text:
    text8 += i

In [48]:
%%time
sentences8 = tokenize_text(text8)

Wall time: 16.9 s


In [49]:
%%time
w2v8 = Word2Vec(sentences=sentences8,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 29.3 s


In [50]:
w2v8.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.931448221206665),
 ('grido', 0.915386438369751),
 ('sgomento', 0.9136419892311096),
 ('raccapriccio', 0.895999014377594),
 ('lacrima', 0.8921935558319092),
 ('sangue', 0.8897198438644409),
 ('orrore', 0.8895189762115479),
 ('sorriso', 0.8833434581756592),
 ('bacio', 0.8766768574714661),
 ('sete', 0.8747053146362305)]

In [51]:
w2v8.save(os.path.join('trained_models/Word2VecCBOW1', '1w2vcbow8.model'))

### Zeitraum 9: 1951-1985

In [52]:
text9 = ''

for i in df9.lemmatized_text:
    text9 += i

In [53]:
%%time
sentences9 = tokenize_text(text9)

Wall time: 18.1 s


In [54]:
%%time
w2v9 = Word2Vec(sentences=sentences9,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 30.5 s


In [55]:
w2v9.wv.most_similar(positive=['terrore'], topn=10)

[('disperato', 0.9205113053321838),
 ('silenzio', 0.9094251990318298),
 ('disperazione', 0.9069213271141052),
 ('noia', 0.8905684947967529),
 ('velo', 0.8886895775794983),
 ('furia', 0.8849733471870422),
 ('orrore', 0.8831459283828735),
 ('veleno', 0.8826703429222107),
 ('urlo', 0.8819399476051331),
 ('urlare', 0.8810703754425049)]

In [56]:
w2v9.save(os.path.join('trained_models/Word2VecCBOW1', '1w2vcbow9.model'))

### Zeitraum 10: 1986-2000

In [57]:
text10 = ''

for i in df10.lemmatized_text:
    text10+= i

In [58]:
%%time
sentences10 = tokenize_text(text10)

Wall time: 35.1 s


In [59]:
%%time
w2v10 = Word2Vec(sentences=sentences10,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 30.1 s


In [60]:
w2v10.wv.most_similar(positive=['terrore'], topn=10)

[('buio', 0.9433283805847168),
 ('giallo', 0.9328383207321167),
 ('freddo', 0.9266476035118103),
 ('sangue', 0.9174508452415466),
 ('barboncino', 0.9162874221801758),
 ('bambina', 0.9160411953926086),
 ('ostia', 0.9158029556274414),
 ('psicolabile', 0.9100673794746399),
 ('impaniare', 0.9094985723495483),
 ('innocente', 0.9065720438957214)]

In [61]:
w2v10.save(os.path.join('trained_models/Word2VecCBOW1', '1w2vcbow10.model'))

### Zeitraum 11: 2001-2021

In [62]:
text11 = ''

for i in df11.lemmatized_text:
    text11+= i

In [63]:
%%time
sentences11 = tokenize_text(text11)

Wall time: 14 s


In [64]:
%%time
w2v11 = Word2Vec(sentences=sentences11,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 30.9 s


In [65]:
w2v11.wv.most_similar(positive=['terrore'], topn=10)

[('tristezza', 0.9850091934204102),
 ('tentazione', 0.9842895269393921),
 ('orecchio', 0.983888566493988),
 ('variconi', 0.9824872612953186),
 ('sgomentare', 0.9816699028015137),
 ('raccapriccio', 0.9775738716125488),
 ('gatto', 0.9755397439002991),
 ('atterrire', 0.9751208424568176),
 ('smorfia', 0.9744898676872253),
 ('mistero', 0.9737504124641418)]

In [66]:
w2v11.save(os.path.join('trained_models/Word2VecCBOW1', '1w2vcbow11.model'))